In [ ]:
# Databricks notebook source
# =============================================================================
# 🏆 MODEL REGISTRATION SCRIPT (READS FROM EVALUATION RESULTS)
# =============================================================================
# Purpose: Register approved models from evaluation pipeline
# Prerequisites: Run model_evaluation_final_fixed.py first
# Fixed: Aliases iteration error completely resolved
# =============================================================================

import mlflow
from mlflow.tracking import MlflowClient
import sys
import os
import requests
import traceback
from typing import Dict, Optional, Tuple, Any
from datetime import datetime
from pyspark.sql import SparkSession
import pandas as pd

print("=" * 80)
print("🏆 MODEL REGISTRATION SYSTEM (FROM EVALUATION RESULTS)")
print("=" * 80)

# ====================== CONFIGURATION ========================= #
class Config:
    """Centralized configuration management"""
    
    def __init__(self):
        # Unity Catalog Configuration
        self.UC_CATALOG = "workspace"
        self.UC_SCHEMA = "ml"
        self.MODEL_NAME = f"{self.UC_CATALOG}.{self.UC_SCHEMA}.house_price_xgboost_uc2"
        
        # Aliases
        self.STAGING_ALIAS = "Staging"
        self.PRODUCTION_ALIAS = "production"
        
        # Delta Tables (must match evaluation script)
        self.BEST_MODEL_METADATA_TABLE = "workspace.default.best_model_metadata"
        self.EVALUATION_LOG_TABLE = "workspace.default.model_evaluation_log"
        
        # Model Configuration
        self.ARTIFACT_PATH = "xgboost_model"
        self.METRIC_KEY = "test_rmse"
        self.TOL = 1e-6  # Float comparison tolerance
        
        # Slack Configuration
        self.SLACK_WEBHOOK_URL = self._get_slack_webhook()
        
    def _get_slack_webhook(self) -> Optional[str]:
        """Safely retrieve Slack webhook URL"""
        try:
            for scope in ["shared-scope", "dev-scope"]:
                try:
                    webhook = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
                    if webhook and webhook.strip():
                        print(f"✅ Slack webhook configured from scope '{scope}'")
                        return webhook
                except Exception:
                    continue
        except Exception:
            pass
        
        print("ℹ️ Slack notifications disabled")
        return None


# Initialize configuration
config = Config()

print("\n📋 CONFIGURATION:")
print(f"   Model Name: {config.MODEL_NAME}")
print(f"   Staging Alias: @{config.STAGING_ALIAS}")
print(f"   Production Alias: @{config.PRODUCTION_ALIAS}")
print(f"   Metadata Table: {config.BEST_MODEL_METADATA_TABLE}")
print(f"   Log Table: {config.EVALUATION_LOG_TABLE}")
print("=" * 80)


# =================== SLACK NOTIFICATION HELPER ==================== #
class SlackNotifier:
    """Enhanced Slack notification handler"""
    
    def __init__(self, webhook_url: Optional[str]):
        self.webhook_url = webhook_url
        self.enabled = webhook_url is not None
        
    def send(self, message: str, level: str = "info") -> bool:
        """Send Slack notification with error handling"""
        if not self.enabled:
            print(f"📢 [SLACK DISABLED] {message}")
            return False
            
        emoji_map = {
            "info": "ℹ️",
            "success": "✅",
            "warning": "⚠️",
            "error": "❌"
        }
        
        formatted_message = f"{emoji_map.get(level, 'ℹ️')} {message}"
        payload = {"text": formatted_message}
        
        try:
            response = requests.post(
                self.webhook_url, 
                json=payload,
                timeout=5
            )
            if response.status_code == 200:
                print(f"📢 Slack notification sent: {level}")
                return True
            else:
                print(f"⚠️ Slack error: {response.status_code}")
                return False
        except Exception as e:
            print(f"❌ Slack notification failed: {e}")
            return False


# Initialize Slack notifier
slack = SlackNotifier(config.SLACK_WEBHOOK_URL)


# =============================================================================
# ✅ INITIALIZATION
# =============================================================================
try:
    spark = SparkSession.builder.appName("ModelRegistration").getOrCreate()
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    print("\n✅ MLflow and Spark initialized")

except Exception as e:
    print(f"❌ Initialization failed: {e}")
    traceback.print_exc()
    sys.exit(1)


# =============================================================================
# 🔧 HELPER: GET MODEL ALIASES SAFELY
# =============================================================================
def get_model_aliases_safe(model_name: str, version: int) -> list:
    """
    Safely get aliases for a model version using direct API call
    Avoids iteration issues with aliases property
    """
    try:
        common_aliases = ['production', 'Staging', 'champion', 'baseline']
        found_aliases = []
        
        for alias in common_aliases:
            try:
                alias_version = client.get_model_version_by_alias(model_name, alias)
                if alias_version and str(alias_version.version) == str(version):
                    found_aliases.append(alias)
            except:
                continue
        
        return found_aliases
    except Exception:
        return []


# =============================================================================
# 📋 STEP 1: READ EVALUATION RESULTS
# =============================================================================
def get_evaluation_results() -> Optional[Dict]:
    """Read latest evaluation results from Delta table"""
    print(f"\n{'='*70}")
    print("📋 STEP 1: Reading Evaluation Results")
    print(f"{'='*70}")

    try:
        # Check if table exists
        tables = spark.catalog.listTables("default")
        table_names = [t.name for t in tables]
        
        if "best_model_metadata" not in table_names:
            print(f"❌ Table '{config.BEST_MODEL_METADATA_TABLE}' not found!")
            print("\n💡 Please run model_evaluation_final_fixed.py first")
            return None
        
        # Read latest evaluation
        df = spark.read.format("delta").table(config.BEST_MODEL_METADATA_TABLE)
        
        if df.count() == 0:
            print("❌ No evaluation results found in table!")
            print("\n💡 Please run model_evaluation_final_fixed.py first")
            return None
        
        # Get latest evaluation (most recent timestamp)
        latest = df.orderBy(df.evaluation_timestamp.desc()).first()
        
        print(f"✅ Evaluation Results Found:")
        print(f"   Evaluated At: {latest.evaluation_timestamp}")
        print(f"   Run ID: {latest.run_id}")
        print(f"   Run Name: {latest.run_name}")
        print(f"   Model URI: {latest.model_uri}")
        print(f"   Metric ({latest.metric_key}): {latest.metric_value:.6f}")
        print(f"   Should Register: {'YES ✅' if latest.should_register else 'NO ❌'}")
        print(f"   Reason: {latest.evaluation_reason}")
        print(f"   Improvement: {latest.improvement_pct:.2f}%")
        print(f"   Total Runs Evaluated: {latest.total_runs_evaluated}")
        
        return {
            'run_id': latest.run_id,
            'run_name': latest.run_name,
            'model_uri': latest.model_uri,
            'artifact_path': latest.artifact_path,
            'metric_key': latest.metric_key,
            'metric_value': float(latest.metric_value),
            'should_register': bool(latest.should_register),
            'reason': latest.evaluation_reason,
            'improvement_pct': float(latest.improvement_pct),
            'total_runs': int(latest.total_runs_evaluated),
            'evaluation_time': latest.evaluation_timestamp,
            'params_json': latest.params_json if hasattr(latest, 'params_json') else "{}"
        }

    except Exception as e:
        print(f"❌ Failed to read evaluation results: {e}")
        traceback.print_exc()
        return None


# =============================================================================
# 🔍 STEP 2: CHECK FOR DUPLICATE VERSIONS (FIXED)
# =============================================================================
def check_duplicate(eval_results: Dict) -> Optional[Any]:
    """Check if model with same run_id already exists - Fixed aliases issue"""
    print(f"\n{'='*70}")
    print("📋 STEP 2: Checking for Duplicates")
    print(f"{'='*70}")

    try:
        mv_list = client.search_model_versions(
            f"name = '{config.MODEL_NAME}'"
        )
        
        # Convert to list safely
        versions_list = []
        try:
            for v in mv_list:
                versions_list.append(v)
        except Exception as e:
            print(f"ℹ️ No existing model versions (first registration)")
            return None
        
    except Exception as e:
        print(f"ℹ️ No existing model versions (first registration)")
        return None
    
    if not versions_list:
        print("ℹ️ No existing versions found (first registration)")
        return None
    
    print(f"✅ Found {len(versions_list)} existing version(s)")
    
    new_run_id = eval_results['run_id']
    new_metric = eval_results['metric_value']
    
    # Check each existing version
    for mv in versions_list:
        try:
            # Check if same run_id
            if mv.run_id == new_run_id:
                # Get aliases safely without iteration
                version_aliases = get_model_aliases_safe(config.MODEL_NAME, mv.version)
                aliases_str = ', '.join(version_aliases) if version_aliases else 'None'
                
                print(f"\n⚠️ DUPLICATE DETECTED!")
                print(f"   Existing Version: v{mv.version}")
                print(f"   Run ID: {mv.run_id}")
                print(f"   Aliases: {aliases_str}")
                print(f"\n   → Model already registered, skipping registration")
                
                slack.send(
                    f"⚠️ Duplicate detected — using existing version *v{mv.version}* "
                    f"for `{config.MODEL_NAME}`",
                    level="warning"
                )
                return mv
            
            # Also check metric similarity (within tolerance)
            try:
                run = client.get_run(mv.run_id)
                old_metric = run.data.metrics.get(config.METRIC_KEY)
                
                if old_metric and abs(old_metric - new_metric) <= config.TOL:
                    print(f"\n⚠️ Similar model found!")
                    print(f"   Version: v{mv.version}")
                    print(f"   Metric difference: {abs(old_metric - new_metric):.8f}")
                    print(f"   (Within tolerance: {config.TOL})")
            except Exception:
                pass
                
        except Exception as e:
            print(f"⚠️ Error checking version {mv.version}: {e}")
            continue
    
    print("\n✅ No duplicates found - proceeding with registration")
    return None


# =============================================================================
# 🚀 STEP 3: REGISTER MODEL TO UNITY CATALOG
# =============================================================================
def register_model(eval_results: Dict) -> Optional[Any]:
    """Register the approved model to Unity Catalog"""
    print(f"\n{'='*70}")
    print("📋 STEP 3: Registering Model to Unity Catalog")
    print(f"{'='*70}")

    # Check if model is approved
    if not eval_results['should_register']:
        print("❌ Model NOT APPROVED for registration")
        print(f"   Reason: {eval_results['reason']}")
        print(f"   Improvement: {eval_results['improvement_pct']:.2f}%")
        
        slack.send(
            f"⏭️ Model registration skipped for `{config.MODEL_NAME}`\n"
            f"Reason: {eval_results['reason']}",
            level="warning"
        )
        return None

    # Check for duplicates
    duplicate = check_duplicate(eval_results)
    if duplicate:
        return duplicate

    # Proceed with registration
    try:
        print(f"\n⏳ Registering model...")
        print(f"   Model URI: {eval_results['model_uri']}")
        print(f"   Target: {config.MODEL_NAME}")
        
        # Register the model
        new_version = mlflow.register_model(
            eval_results['model_uri'], 
            config.MODEL_NAME
        )
        
        print(f"\n{'='*70}")
        print("✅ MODEL REGISTERED SUCCESSFULLY!")
        print(f"{'='*70}")
        print(f"   Model Name: {config.MODEL_NAME}")
        print(f"   Version: v{new_version.version}")
        print(f"   Source Run ID: {eval_results['run_id']}")
        print(f"   Run Name: {eval_results['run_name']}")
        print(f"   {config.METRIC_KEY}: {eval_results['metric_value']:.6f}")
        print(f"   Improvement: {eval_results['improvement_pct']:.2f}%")
        print(f"{'='*70}\n")
        
        # Send Slack notification
        slack.send(
            f"✅ Model *{config.MODEL_NAME}* registered as version *v{new_version.version}*\n"
            f"📊 {config.METRIC_KEY}: {eval_results['metric_value']:.6f}\n"
            f"📈 Improvement: {eval_results['improvement_pct']:.2f}%\n"
            f"🔗 Run: {eval_results['run_name']}",
            level="success"
        )
        
        return new_version

    except Exception as e:
        print(f"❌ Registration failed: {e}")
        traceback.print_exc()
        
        slack.send(
            f"❌ Model registration failed for `{config.MODEL_NAME}`: {e}",
            level="error"
        )
        return None


# =============================================================================
# 🏷️ STEP 4: SET STAGING ALIAS & ADD TAGS
# =============================================================================
def set_staging_alias_and_tags(version_number: int, eval_results: Dict) -> bool:
    """Set Staging alias and add comprehensive tags"""
    print(f"\n{'='*70}")
    print("📋 STEP 4: Setting Alias & Tags")
    print(f"{'='*70}")

    try:
        # Set Staging alias
        print(f"⏳ Setting @{config.STAGING_ALIAS} alias for Version {version_number}")
        
        client.set_registered_model_alias(
            config.MODEL_NAME, 
            config.STAGING_ALIAS, 
            version_number
        )
        
        print(f"✅ Alias Set: Version {version_number} → @{config.STAGING_ALIAS}")
        
        # Add comprehensive tags
        tags = {
            "registered_from": "registration_pipeline",
            "evaluation_reason": eval_results['reason'],
            "improvement_pct": f"{eval_results['improvement_pct']:.2f}",
            "registration_timestamp": datetime.now().isoformat(),
            "metric_rmse": str(eval_results['metric_value']),
            "source_run_id": eval_results['run_id'],
            "source_run_name": eval_results['run_name'],
            "total_runs_evaluated": str(eval_results['total_runs']),
            "artifact_path": eval_results['artifact_path'],
            "evaluation_timestamp": str(eval_results['evaluation_time'])
        }
        
        print(f"\n🏷️  Adding tags to Version {version_number}:")
        for key, value in tags.items():
            try:
                client.set_model_version_tag(
                    config.MODEL_NAME, 
                    version_number, 
                    key, 
                    value
                )
                print(f"   ✅ {key}: {value}")
            except Exception as e:
                print(f"   ⚠️ Failed to add tag {key}: {e}")
        
        print(f"\n✅ All tags added successfully")
        return True

    except Exception as e:
        print(f"❌ Failed to set alias or tags: {e}")
        traceback.print_exc()
        return False


# =============================================================================
# 📝 STEP 5: UPDATE EVALUATION LOG
# =============================================================================
def update_evaluation_log(version_number: int, eval_results: Dict) -> bool:
    """Update evaluation log with registration status"""
    print(f"\n{'='*70}")
    print("📋 STEP 5: Updating Evaluation Log")
    print(f"{'='*70}")

    try:
        # Read existing log
        df = spark.read.format("delta").table(config.EVALUATION_LOG_TABLE)
        
        if df.count() == 0:
            print("⚠️ Evaluation log is empty, cannot update")
            return False
        
        # Convert to pandas for easier manipulation
        pdf = df.toPandas()
        
        # Find the matching row by run_id
        mask = pdf['new_run_id'] == eval_results['run_id']
        
        if mask.any():
            pdf.loc[mask, 'promoted_to_staging'] = True
            pdf.loc[mask, 'promoted_version'] = int(version_number)
            
            # Convert back to Spark DataFrame
            updated_df = spark.createDataFrame(pdf)
            
            # Overwrite table
            updated_df.write.format("delta")\
                .mode("overwrite")\
                .option("overwriteSchema", "true")\
                .saveAsTable(config.EVALUATION_LOG_TABLE)
            
            print(f"✅ Evaluation log updated")
            print(f"   Run ID: {eval_results['run_id']}")
            print(f"   Marked as promoted to Staging")
            print(f"   Version: v{version_number}")
            return True
        else:
            print(f"⚠️ No matching record found for run_id: {eval_results['run_id']}")
            return False

    except Exception as e:
        print(f"⚠️ Failed to update evaluation log: {e}")
        print("   (Non-critical error - continuing)")
        return False


# =============================================================================
# 📊 STEP 6: DISPLAY REGISTRATION SUMMARY
# =============================================================================
def display_summary(eval_results: Dict, version_number: int) -> None:
    """Display comprehensive registration summary"""
    print(f"\n{'='*80}")
    print("✅ MODEL REGISTRATION COMPLETE")
    print(f"{'='*80}")
    
    print(f"\n📊 Source Model:")
    print(f"   Run ID: {eval_results['run_id']}")
    print(f"   Run Name: {eval_results['run_name']}")
    print(f"   RMSE: {eval_results['metric_value']:.6f}")
    print(f"   Rank: #1 from {eval_results['total_runs']} runs")
    print(f"   Evaluated At: {eval_results['evaluation_time']}")
    
    print(f"\n🏆 Registered Model:")
    print(f"   Model Name: {config.MODEL_NAME}")
    print(f"   Version: v{version_number}")
    print(f"   Alias: @{config.STAGING_ALIAS}")
    print(f"   Status: ✅ ACTIVE")
    
    print(f"\n📈 Performance:")
    print(f"   Improvement: {eval_results['improvement_pct']:.2f}%")
    print(f"   Reason: {eval_results['reason']}")
    
    print(f"\n🎯 Next Steps:")
    print(f"   1. Test model in Staging environment")
    print(f"   2. Validate performance metrics")
    print(f"   3. Run A/B tests if applicable")
    print(f"   4. Promote to @{config.PRODUCTION_ALIAS} if successful")
    
    print(f"\n📦 Model Access:")
    print(f"   UC Path: {config.MODEL_NAME}")
    print(f"   Alias: models:/{config.MODEL_NAME}@{config.STAGING_ALIAS}")
    print(f"   Version: models:/{config.MODEL_NAME}/{version_number}")
    
    print("\n" + "=" * 80)


# =============================================================================
# 🎬 MAIN EXECUTION
# =============================================================================
def main():
    """Main execution flow with comprehensive error handling"""
    print(f"\n{'='*80}")
    print("🚀 STARTING MODEL REGISTRATION PIPELINE")
    print(f"{'='*80}")
    
    try:
        # Step 1: Read evaluation results
        eval_results = get_evaluation_results()
        if not eval_results:
            print("\n❌ REGISTRATION FAILED - No evaluation results found")
            print("\n💡 Please run model_evaluation_final_fixed.py first")
            sys.exit(1)
        
        # Check approval status
        if not eval_results['should_register']:
            print(f"\n{'='*80}")
            print("⏭️ REGISTRATION SKIPPED")
            print(f"{'='*80}")
            print(f"   Model was NOT approved during evaluation")
            print(f"   Reason: {eval_results['reason']}")
            print(f"   Improvement: {eval_results['improvement_pct']:.2f}%")
            print(f"\n💡 Model needs better performance to be registered")
            print(f"{'='*80}")
            sys.exit(0)  # Exit gracefully, not an error
        
        # Step 2 & 3: Register model (includes duplicate check)
        new_version = register_model(eval_results)
        if not new_version:
            print("\n❌ REGISTRATION FAILED")
            sys.exit(1)
        
        # Step 4: Set Staging alias and add tags
        alias_set = set_staging_alias_and_tags(new_version.version, eval_results)
        if not alias_set:
            print("\n⚠️ WARNING: Model registered but alias/tags not set properly")
        
        # Step 5: Update evaluation log
        update_evaluation_log(new_version.version, eval_results)
        
        # Step 6: Display summary
        display_summary(eval_results, new_version.version)
        
        print("✅ Registration pipeline completed successfully!")
        sys.exit(0)
        
    except KeyboardInterrupt:
        print("\n\n⚠️ Process interrupted by user")
        slack.send(
            f"⚠️ Model registration interrupted for `{config.MODEL_NAME}`",
            level="warning"
        )
        sys.exit(1)
        
    except Exception as e:
        print(f"\n❌ UNEXPECTED ERROR: {e}")
        traceback.print_exc()
        
        slack.send(
            f"❌ Critical error in model registration: {e}",
            level="error"
        )
        sys.exit(1)


# ============================ ENTRY POINT ============================ #
if __name__ == "__main__":
    main()





# # Databricks notebook source
# import mlflow
# from mlflow.tracking import MlflowClient
# import sys
# import os
# import requests  # ✅ Added for Slack webhook notifications

# # ====================== CONFIGURATION ========================= #
# try:
#     EXPERIMENT_NAME = dbutils.widgets.get("experiment_name")
#     print(f"✓ Experiment Name from widget: {EXPERIMENT_NAME}")
# except:
#     EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
#     print(f"ℹ Using default experiment: {EXPERIMENT_NAME}")

# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"

# # =================== MODEL CONFIG METADATA ==================== #
# MODEL_CONFIG = {
#     "xgboost": {
#         "model_name": "house_price_xgboost_uc2",
#         "artifact_path": "xgboost_model",
#         "param_keys": [
#             "n_estimators", "max_depth", "learning_rate",
#             "subsample", "colsample_bytree"
#         ],
#         "metric_key": "test_rmse",
#         "keywords": ["xgboost", "xgb", "house_price", "config_runs"]
#     }
# }

# # =================== SLACK NOTIFICATION HELPER ==================== #
# def send_slack_notification(message):
#     """
#     Send Slack message using webhook URL.
#     Set SLACK_WEBHOOK_URL as environment variable or Databricks secret.
#     """
#     webhook_url = os.getenv("SLACK_WEBHOOK_URL")
#     if not webhook_url:
#         print("⚠️ No Slack webhook URL found. Skipping notification.")
#         return

#     payload = {"text": message}
#     try:
#         response = requests.post(webhook_url, json=payload)
#         if response.status_code == 200:
#             print("📢 Slack notification sent successfully.")
#         else:
#             print(f"⚠️ Slack notification failed: {response.status_code}, {response.text}")
#     except Exception as e:
#         print(f"❌ Slack notification error: {e}")

# # ================== MODEL TYPE DETECTION ====================== #
# def detect_model_config(experiment_name: str):
#     exp_lower = experiment_name.lower()
#     for model_type, cfg in MODEL_CONFIG.items():
#         for key in cfg["keywords"]:
#             if key in exp_lower:
#                 full_uc_name = f"{UC_CATALOG}.{UC_SCHEMA}.{cfg['model_name']}"
#                 print(f"✓ Detected model type: {model_type.upper()}")
#                 print(f"✓ UC Model Name: {full_uc_name}")
#                 return (
#                     full_uc_name,
#                     cfg["artifact_path"],
#                     cfg["param_keys"],
#                     cfg["metric_key"]
#                 )
#     raise ValueError("❌ No matching model config found based on experiment name!")

# REGISTERED_MODEL_NAME, ARTIFACT_PATH, PARAM_KEYS, METRIC_KEY = detect_model_config(EXPERIMENT_NAME)
# TOL = 1e-6  # float tolerance

# # ====================== UTILITIES ====================== #
# def normalize(val):
#     try:
#         if '.' not in str(val) and str(val).isdigit():
#             return int(val)
#         return float(val)
#     except:
#         return str(val)

# # ================== FIND BEST RUN ====================== #
# def get_best_run(client):
#     exp = client.get_experiment_by_name(EXPERIMENT_NAME)
#     if not exp:
#         print("❌ Experiment not found.")
#         return None, {}, {}

#     runs = client.search_runs(
#         [exp.experiment_id], 
#         order_by=[f"metrics.{METRIC_KEY} ASC"],
#         max_results=1000
#     )
    
#     if not runs:
#         print("⚠ No runs found in experiment.")
#         return None, {}, {}

#     best_run = None
#     best_metric = float("inf")

#     for r in runs:
#         metric_val = r.data.metrics.get(METRIC_KEY)
#         if metric_val is not None:
#             print(f"  📊 Run: {r.info.run_name or r.info.run_id[:8]} | {METRIC_KEY}: {metric_val:.4f}")
#             if metric_val < best_metric:
#                 best_metric = metric_val
#                 best_run = r

#     if best_run:
#         params = {k: normalize(v) for k, v in best_run.data.params.items() if k in PARAM_KEYS}
#         metrics = best_run.data.metrics
        
#         print(f"\n{'='*70}")
#         print(f"🏆 BEST RUN IDENTIFIED:")
#         print(f"{'='*70}")
#         print(f"   Run Name: {best_run.info.run_name or 'N/A'}")
#         print(f"   Run ID: {best_run.info.run_id}")
#         print(f"   {METRIC_KEY}: {best_metric:.6f}")
#         print(f"   Parameters: {params}")
#         print(f"{'='*70}\n")
        
#         return best_run.info.run_id, params, metrics
#     else:
#         print("⚠ No valid runs with metric found.")
#         return None, {}, {}

# # ================ DUPLICATE VERSION CHECK ===================== #
# def check_duplicate(client, new_params, new_metrics):
#     try:
#         mv_list = client.search_model_versions(f"name = '{REGISTERED_MODEL_NAME}'")
#     except Exception as e:
#         print(f"ℹ No existing model versions found (this may be first registration): {e}")
#         return None
    
#     if not mv_list:
#         return None

#     new_metric_val = new_metrics.get(METRIC_KEY, None)
    
#     for mv in mv_list:
#         try:
#             run = client.get_run(mv.run_id)
#         except Exception as e:
#             print(f"⚠ Could not fetch run {mv.run_id}: {e}")
#             continue

#         old_params = {k: normalize(v) for k, v in run.data.params.items() if k in new_params}
#         old_metric_val = run.data.metrics.get(METRIC_KEY, None)

#         same_params = all(old_params.get(k) == new_params.get(k) for k in new_params)
#         same_metric = (
#             old_metric_val is not None and new_metric_val is not None
#             and abs(old_metric_val - new_metric_val) <= TOL
#         )
        
#         if same_params and same_metric:
#             print(f"\n⏭️ DUPLICATE DETECTED!")
#             print(f"   Existing Version: {mv.version}")
#             print(f"   Run ID: {mv.run_id}")
#             print(f"   This model is already registered with same params & performance.")
#             return mv
            
#     return None

# # ================== REGISTER MODEL LOGIC ======================= #
# def register_model(client, run_id, params, metrics):
#     duplicate_version = check_duplicate(client, params, metrics)
#     if duplicate_version:
#         print(f"✅ Using existing registered version: {duplicate_version.version}")
#         send_slack_notification(f"⚠️ Duplicate detected — using existing model version *v{duplicate_version.version}* for `{REGISTERED_MODEL_NAME}`.")
#         return duplicate_version

#     model_uri = f"runs:/{run_id}/{ARTIFACT_PATH}"
#     print(f"\n⏳ Registering new model version...")
#     print(f"   Model URI: {model_uri}")
#     print(f"   Target: {REGISTERED_MODEL_NAME}")

#     try:
#         new_version = mlflow.register_model(model_uri, REGISTERED_MODEL_NAME)
        
#         print(f"\n{'='*70}")
#         print("✅ MODEL REGISTERED SUCCESSFULLY!")
#         print(f"{'='*70}")
#         print(f"   Model Name: {REGISTERED_MODEL_NAME}")
#         print(f"   Version: {new_version.version}")
#         print(f"   Source Run ID: {run_id}")
#         print(f"   {METRIC_KEY}: {metrics.get(METRIC_KEY, 'N/A')}")
#         print(f"{'='*70}\n")

#         # 🏷️ Add helpful tags
#         client.set_model_version_tag(
#             REGISTERED_MODEL_NAME, 
#             new_version.version, 
#             "source_run_id", 
#             run_id
#         )
#         client.set_model_version_tag(
#             REGISTERED_MODEL_NAME, 
#             new_version.version, 
#             "experiment_name", 
#             EXPERIMENT_NAME
#         )
#         client.set_model_version_tag(
#             REGISTERED_MODEL_NAME, 
#             new_version.version, 
#             "metric_rmse", 
#             str(metrics.get(METRIC_KEY, ""))
#         )
        
#         for param_key, param_val in params.items():
#             client.set_model_version_tag(
#                 REGISTERED_MODEL_NAME,
#                 new_version.version,
#                 f"param_{param_key}",
#                 str(param_val)
#             )

#         # ✅ Send Slack success message
#         send_slack_notification(
#             f"✅ Model *{REGISTERED_MODEL_NAME}* registered successfully as version *v{new_version.version}*.\n"
#             f"🏷 Metric `{METRIC_KEY}` = {metrics.get(METRIC_KEY, 'N/A')}\n"
#             f"🔗 Run ID: {run_id}"
#         )

#         return new_version
        
#     except Exception as e:
#         print(f"❌ Registration Failed: {e}")
#         send_slack_notification(f"❌ Model registration failed for `{REGISTERED_MODEL_NAME}` — {e}")
#         import traceback
#         traceback.print_exc()
#         sys.exit(1)

# # ============================ MAIN ============================ #
# if __name__ == "__main__":
#     print("\n" + "=" * 70)
#     print("🚀 MODEL REGISTRATION - BEST RUN SELECTION (MLflow + UC)")
#     print("=" * 70 + "\n")

#     client = MlflowClient()
    
#     print(f"📋 Configuration:")
#     print(f"   Experiment: {EXPERIMENT_NAME}")
#     print(f"   Target Model: {REGISTERED_MODEL_NAME}")
#     print(f"   Metric to optimize: {METRIC_KEY} (lower is better)")
#     print(f"   Artifact Path: {ARTIFACT_PATH}\n")
    
#     print("🔍 Searching for best run...")
#     run_id, params, metrics = get_best_run(client)

#     if run_id:
#         register_model(client, run_id, params, metrics)
#         print("\n✨ Registration process completed successfully!")
#     else:
#         print("❌ No valid best run found. Exiting.")
#         send_slack_notification(f"⚠️ Model registration skipped — no valid best run found for `{REGISTERED_MODEL_NAME}`.")
#         sys.exit(1)
